
# RAG con PDFs (Notebook paralelo)
Este cuaderno replica el flujo de `rag_cli.py` pero muestra cada etapa con resúmenes y estadísticas para entender los datos procesados.


In [2]:
pip install -r requirements.txt

  Using cached langchainhub-0.1.21-py3-none-any.whl.metadata (659 bytes)
  Using cached langchain_qdrant-1.1.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached langchain_experimental-0.4.1-py3-none-any.whl.metadata (1.3 kB)
  Using cached pypdf-6.5.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached qdrant_client-1.16.2-py3-none-any.whl.metadata (11 kB)
  Using cached faiss_cpu-1.13.1-cp310-abi3-macosx_14_0_arm64.whl.metadata (7.6 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached beautifulsoup4-4.14.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached types_requests-2.32.4.20250913-py3-none-any.whl.metadata (2.0 kB)
  Usi

In [8]:

from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from rag_data_preparation import (
    PDF_DIR,
    load_pdf_documents,
    extract_metadata,
    chunk_documents,
    build_vector_store,
)
from rag_cli import (
    build_query_rewriter,
    answer_question,
)

load_dotenv()
print(f"📁 Directorio de PDFs: {PDF_DIR.resolve()}")


📁 Directorio de PDFs: /Users/fabiansalfatenavarro/Desktop/Diplomando IA/Archivos/M3/Tarea Grupal/pdf


## Inventario de PDFs

In [9]:

pdf_records = []
for path in sorted(PDF_DIR.glob('*.pdf')):
    stat = path.stat()
    pdf_records.append({
        "archivo": path.name,
        "tam_mb": round(stat.st_size / 1e6, 2),
        "modificado": stat.st_mtime,
    })

pdf_df = pd.DataFrame(pdf_records)
if pdf_df.empty:
    raise RuntimeError("No hay PDFs en la carpeta pdf/.")
pdf_df


,archivo,tam_mb,modificado
0,Estudio-Claves-para-hacer-negocios-con-Japon-2...,5.05,1.766465e+09
1,Manual-de-Exportacion-de-Servicios-2025.pdf,29.26,1.766465e+09
2,NoCobre_NoLitio_noviembre-1.pdf,1.68,1.766465e+09


## 1. Carga completa de documentos

In [10]:
raw_documents = load_pdf_documents()
print(f"Documentos cargados: {len(raw_documents)}")
print("\nEjemplo de metadata base:")
print(raw_documents[0].metadata)
print("\nFragmento del contenido:\n", raw_documents[0].page_content[:500])


Documentos cargados: 3

Ejemplo de metadata base:
{'source': 'pdf/Estudio-Claves-para-hacer-negocios-con-Japon-2025.pdf'}

Fragmento del contenido:
 Claves parahacer negocios conJapón | 2025Oficina Comercial de ProChile en Tokyo, JapónProChile - Todos los derechos reservados.Marzo, 2025

1. Evolución del empresario en el mercado actual
Claves para hacer negocios con Japón | 2025Características del empresarioUna de las características más comunes del empresario japones, es el tiempo que necesita para cerrar un negocio.La toma de decisiones del empresario japonés es normalmente muy lenta, requiriendo bastante tiempo, y una gran cantidad de inf


## 2. Extracción de metadata con GPT-4o

In [11]:

llm = ChatOpenAI(model='gpt-4o', temperature=0.1)
metadata_map = extract_metadata(raw_documents, llm)
metadata_df = pd.DataFrame([
    {
        'source': src,
        'titulo': meta.titulo,
        'categoria': meta.categoria,
        'resumen': meta.resumen,
    }
    for src, meta in metadata_map.items()
])
metadata_df


,source,titulo,categoria,resumen
0,pdf/Estudio-Claves-para-hacer-negocios-con-Jap...,Claves para Negocios con Japón en 2025,Negocios Internacionales,El empresario japonés se caracteriza por su pr...
1,pdf/Manual-de-Exportacion-de-Servicios-2025.pdf,Manual de Exportación de Servicios,Comercio Internacional,Este manual ofrece una guía completa sobre el ...
2,pdf/NoCobre_NoLitio_noviembre-1.pdf,Informe de Exportaciones No Cobre No Litio - N...,Economía y Comercio Exterior,Este informe analiza las exportaciones chilena...


## 3. Semantic chunking y estadísticas

In [12]:
chunk_embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
chunker = SemanticChunker(
    chunk_embeddings,
    breakpoint_threshold_type='percentile',
    breakpoint_threshold_amount=50,
)
chunked_docs = chunk_documents(raw_documents, metadata_map, chunker)
print(f"Chunks generados: {len(chunked_docs)}")

stats_df = pd.DataFrame([
    {
        'titulo': doc.metadata.get('titulo', 'N/A'),
        'source': doc.metadata.get('source'),
        'chars': len(doc.page_content),
        'words': len(doc.page_content.split()),
    }
    for doc in chunked_docs
])
print("\nResumen por documento:")
stats_summary = stats_df.groupby(['titulo']).agg(
    chunks=('source', 'count'),
    palabras_promedio=('words', 'mean'),
)
stats_summary


Chunks generados: 623

Resumen por documento:


,chunks,palabras_promedio
titulo,,
Claves para Negocios con Japón en 2025,48,80.583333
Informe de Exportaciones No Cobre No Litio - Noviembre 2025,34,225.911765
Manual de Exportación de Servicios,541,52.329020


## 4. Creación / actualización del vector store en Qdrant

In [13]:

vector_store = build_vector_store(chunked_docs)
print('Vector store actualizado en Qdrant. Colección:', vector_store.collection_name)


Vector store actualizado en Qdrant. Colección: rag_mod3_pdf_exportaciones


### Estado de la colección en Qdrant

In [14]:
import os
from qdrant_client import QdrantClient

client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
)
collection = client.get_collection("rag_mod3_pdf_exportaciones")
params = collection.config.params
vector_size = None
if hasattr(params, "vectors") and isinstance(params.vectors, dict):
    default_vec = params.vectors.get("")
    if default_vec is not None:
        vector_size = getattr(default_vec, "size", None)
elif isinstance(params, dict):
    vectors = params.get("vectors", {})
    if isinstance(vectors, dict):
        default_vec = vectors.get("")
        if isinstance(default_vec, dict):
            vector_size = default_vec.get("size")

print("Vectores indexados:", collection.points_count)
print("Dimensiones:", vector_size)


Vectores indexados: 623
Dimensiones: 3072


## 5. Consulta de ejemplo con reescritura

In [15]:

rewrite_chain = build_query_rewriter(llm)
demo_query = "¿Qué oportunidades comerciales hay para exportar servicios en 2025?"
response = answer_question(demo_query, vector_store, llm, rewrite_chain)
print(response)


En 2025, las oportunidades para exportar servicios desde Chile son diversas y abarcan varios sectores. Según el "Informe de Exportaciones No Cobre No Litio - Noviembre 2025", los principales servicios exportados incluyen tecnologías de la información y comunicación (TICs), mantenimiento y reparación, administración, servicios financieros, investigación y desarrollo, asesoría, ingeniería, transporte, y servicios logísticos, entre otros. Estos sectores muestran variaciones en sus exportaciones, con TICs liderando con un valor FOB de US$ 909 millones y un crecimiento del 2,7% en comparación con el año anterior.

Además, el "Manual de Exportación de Servicios" sugiere que los servicios representan una alternativa para diversificar las canastas exportadoras en áreas de alto valor agregado e inserción en cadenas globales de valor. Esto implica que hay un potencial significativo para que las empresas chilenas expandan su oferta de servicios a mercados internacionales, aprovechando la capacida